# Custom Function Call

## 01. Importación de librerías

In [ ]:
import os
import json
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai import OpenAI

## 02. Variables de Entorno

In [ ]:
# load the environment variables
load_dotenv()

# get the API key from the environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GPT4O_MODEL = os.getenv("OPENAI_GPT4O_MODEL")
GPT35_MODEL = os.getenv("OPENAI_GPT35_MODEL")

# print the API key and model
print(f"API Key: {OPENAI_API_KEY[:4]}...{OPENAI_API_KEY[-4:]}")
print(f"Model: {GPT4O_MODEL}")
print(f"Model: {GPT35_MODEL}")

## 03. Cliente OpenAI

In [ ]:
client = OpenAI(api_key=OPENAI_API_KEY)

## 04. Funcion Chat Completion Request

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, model=GPT4O_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

## 05. Definicion de Tools

In [ ]:
sumar_scheme = {
    "type": "function",
    "function": {
        "name": "sumar",
        "description": "Sumar dos números",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "Primer número",
                },
                "b": {
                    "type": "integer",
                    "description": "Segundo número",
                },
            },
            "required": ["a", "b"],
        },
    }
}


In [ ]:

multiplicar_scheme = {
    "type": "function",
    "function": {
        "name": "multiplicar",
        "description": "Multiplicar dos números",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "Primer número",
                },
                "b": {
                    "type": "integer",
                    "description": "Segundo número",
                },
            },
            "required": ["a", "b"],
        },
    }
}

In [ ]:
tools_str = json.dumps([sumar_scheme, multiplicar_scheme], indent=2)
print(tools_str)

## 06. Custom Function Call

In [ ]:
system_prompt = """You only reply in JSON format. 
You are given a set of [TOOLS] to work with. Each tool has a name, a description, and a set of parameters.
Each parameter will have a type and a description.
The required parameters are listed in the 'required' field. If a parameter is not in this list, it is optional.
Your objective is to choose one or more tools, and provide the required parameters for each tool, to solve the given problem.
If no tools are needed, you can provide an empty list.

Given the user [QUERY], you must return a JSON object with the following structure:
[
    {
        "name": "TOOL_NAME",
        "arguments": {
            "ARGUMENT_NAME": ARGUMENT_VALUE,
            ...
        }
    },
    ...
]

Note: Do not enclose the JSON output in a code block. e.g. Do not use triple backticks.
"""

In [ ]:
a = 1354
b = 58751

In [ ]:
query = f"Cuanto es {a} + {b}?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"[TOOLS]\n{tools_str}\n[QUERY]\n{query}"}
]
response = chat_completion_request(messages)
content = response.choices[0].message.content

print(f"Query: {query}")
print(f"Response: {content}")

## 08. Ejecución de Function Call

In [ ]:
def sumar(a: int, b: int) -> int:
    return a + b

def multiplicar(a: int, b: int) -> int:
    return a * b

def execute_tool(tool_name, parameters):
    parameters = json.loads(parameters)
    if tool_name == "sumar":
        result = sumar(parameters["a"], parameters["b"])
    elif tool_name == "multiplicar":
        result = multiplicar(parameters["a"], parameters["b"])
    return result

In [ ]:
query = f"Cuanto es {a} + {b}?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"[TOOLS]\n{tools_str}\n[QUERY]\n{query}"}
]
response = chat_completion_request(messages)
content = response.choices[0].message.content
json_content = json.loads(content)

print(f"Query: {query}")
for tool in json_content:
    tool_result = execute_tool(tool["name"], json.dumps(tool["arguments"]))
    print(f"{tool["name"]}({json.dumps(tool["arguments"])}) = {tool_result}")


## 07. Custom Function Call con Dependencias

In [ ]:
system_prompt = """You only reply in JSON format. 
You are given a set of [TOOLS] to work with. Each tool has a name, a description, and a set of parameters.
Each parameter will have a type and a description.
The required parameters are listed in the 'required' field. If a parameter is not in this list, it is optional.
Your objective is to choose one or more tools, and provide the required parameters for each tool, to solve the given problem.
If the problem requires a multi step solution, you can provide multiple tools in the list. 
If a tool requires the output of a previous tool, you must declare the dependency in the 'dependencies' field.
If no tools are needed, you can provide an empty list.

Given the user [QUERY], you must return a JSON object with the following structure:
[
    {
        "task_id": "TASK_ID",
        "name": "TOOL_NAME",
        "arguments": {
            "ARGUMENT_NAME": ARGUMENT_VALUE/DEPENDENCY_TASK_ID,
            ...
        }
        "dependencies": ["DEPENDENCY_TASK_ID", ...]
    },
    ...
]

Note: Do not enclose the JSON output in a code block. e.g. Do not use triple backticks.
"""

In [ ]:
query = f"Cuanto es ({a} + {b}) * {b}?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"[TOOLS]\n{tools_str}\n[QUERY]\n{query}"}
]
response = chat_completion_request(messages)
content = response.choices[0].message.content

print(f"Query: {query}")
print(f"Response: {content}")